In [1]:
from urllib.request import urlopen
from pandas import json_normalize
from tqdm import tqdm, trange
from datetime import datetime, timedelta
from sqlalchemy import types, create_engine
from tqdm import tqdm 

import pandas as pd
import json
import time
import os
import cx_Oracle
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
## JSON 호출

def get_json(dep_city, arr_city, date, airline):
    url = (
        'https://domair.interpark.com/api/booking/airJourney.do?format=json&dep=' 
        + dep_city
        + '&arr=' 
        + arr_city
        + '&depDate=' 
        + date 
        +'&adt=1&chd=0&inf=0&tripDivi=0&airlineCode=' 
        + airline 
        + '&siteCode='
    )
    
    try:
        json_data = json.load(urlopen(url))

        if json_data["replyHeader"]["errorCode"] == "0":
            return json_data
    
    except Exception as ex:

        print("Error code: ", ex)
        print("Sleep a min & Retry ")

        time.sleep(60)

        get_json(dep_city, arr_city, date, airline) # 회귀

In [3]:
start_time = datetime.now()
print("Start: ", start_time.strftime('%Y-%m-%d %H:%M'))

Start:  2023-08-18 15:32


In [4]:
airport_5 = ['GMP', 'PUS', 'CJJ', 'KWJ', 'TAE']

air_route = []

for depart in airport_5:
    # for arrive in jeju:
    air_route.append([depart, 'CJU'])
    air_route.append(['CJU', depart])

airline_list = ['KE', 'OZ', '7C', 'LJ', 'TW', 'BX', 'RS', 'ZE']

In [5]:
final_df = pd.DataFrame()

search_date = datetime.today().strftime('%Y%m%d')

folder_path = os.path.join(os.getcwd(), search_date)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for i in trange(1, 4):
    
    depart_date = datetime.today() + timedelta(days = i)
    depart_date = depart_date.strftime('%Y%m%d')
    
    for airline in airline_list:

        for line in air_route:

            json_data = get_json(line[0], line[1], depart_date, airline)

            file_name = f"{depart_date}_{line[0]}_{line[1]}_{airline}.json"

            file_path = os.path.join(os.getcwd(), search_date, file_name)

            with open(file_path, 'w', encoding='utf-8-sig') as file:
                json.dump(json_data, file, indent="\t")

 67%|██████▋   | 2/3 [01:20<00:40, 40.01s/it]


KeyboardInterrupt: 

In [ ]:
end_time = datetime.now()

print("End: ", end_time.strftime('%Y-%m-%d %H:%M'))
print("Running time: ", end_time - start_time)